In [ ]:
%%sh
apt-get update
apt-get -y install graphviz
pip install -q graphviz

In [ ]:
import pandas as pd

dataset = pd.read_csv('housing.csv')

In [ ]:
# Move 'medv' column to front
dataset = pd.concat([dataset['medv'], dataset.drop(['medv'], axis=1)], axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
training_dataset, validation_dataset = train_test_split(dataset, test_size=0.1)

print(training_dataset.shape)
print(validation_dataset.shape)

In [ ]:
training_dataset.to_csv('training_dataset.csv', index=False, header=False)
validation_dataset.to_csv('validation_dataset.csv', index=False, header=False)

In [ ]:
import sagemaker

print(sagemaker.__version__)

sess = sagemaker.Session()
bucket = sess.default_bucket()

prefix = 'boston-housing'
training_data_path = sess.upload_data(path='training_dataset.csv', key_prefix=prefix + '/input/training')
validation_data_path = sess.upload_data(path='validation_dataset.csv', key_prefix=prefix + '/input/validation')

print(training_data_path)
print(validation_data_path)

In [ ]:
from sagemaker import get_execution_role
from sagemaker.image_uris import retrieve
from sagemaker.estimator import Estimator

region = sess.boto_session.region_name    
container = retrieve('linear-learner', region)
print(container)

ll_estimator = Estimator(container,
    role=get_execution_role() , 
    instance_count=1,
    instance_type='ml.m5.large',
    output_path='s3://{}/{}/output'.format(bucket, prefix)
)

ll_estimator.set_hyperparameters(
    predictor_type='regressor', 
    mini_batch_size=32)

In [ ]:
from sagemaker import TrainingInput

training_data_channel   = TrainingInput(s3_data=training_data_path, content_type='text/csv')
validation_data_channel = TrainingInput(s3_data=validation_data_path, content_type='text/csv')

ll_data = {'train': training_data_channel, 'validation': validation_data_channel}

In [ ]:
ll_estimator.fit(ll_data)

In [ ]:
%%bash -s $ll_estimator.model_data
aws s3 cp $1 .
tar xvfz model.tar.gz
unzip -o model_algo-1

In [ ]:
import json

sym_json = json.load(open('mx-mod-symbol.json'))
sym_json_string = json.dumps(sym_json)

In [ ]:
import mxnet as mx
from mxnet import gluon

net = gluon.nn.SymbolBlock(
    outputs=mx.sym.load_json(sym_json_string),
    inputs=mx.sym.var('data')) 

In [ ]:
mx.viz.plot_network(
    net(mx.sym.var('data'))[0],   
    node_attrs={'shape':'oval','fixedsize':'false'})

In [ ]:
net.load_parameters('mx-mod-0000.params', allow_missing=True)
net.collect_params().initialize()

In [ ]:
test_sample = mx.nd.array([0.00632,18.00,2.310,0,0.5380,6.5750,65.20,4.0900,1,296.0,15.30,4.98])

In [ ]:
test_sample

In [ ]:
response = net(test_sample)
print(response)